<a href="https://colab.research.google.com/github/roarkai/pytorch_notes/blob/master/Autograd_mechanism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autograd

## I. Autograd记录运算历史的方式
1. Autograd是一个自动微分系统。从概念上看,在执行forward pass中的函数运算的同时，autograd会记录一个有向无环图(DAG),这个图上记录了所有执行过的operation。这个图上,leaf是input tensor,root是output tensor。在执行backward pass时，通过从root到leaf的方向tracing此图,就可以用链式法则自动计算梯度。

2. **从具体的执行来看，autograd用Function objects graph来表达上述DAG。**

  (1) tensor使用的function是**torch.autograd.Function**的实例。这些function class中都定义了forward和backward函数。用他们对tensor做运算时，会自动创建计算图。

  (2) 可以apply()Function objects graph来计算评估图的结果。

  (3) 计算Forward pass时,autograd在执行对应的function的同时，还会构建一个graph来表示这些将要计算梯度的function.每个torch.Tensor的.grad_fn属性都是进入此图的入口。完成Forward pass后,就可以在反向传播中evaluate the graph以计算梯度。

3. **每个迭代都会从零开始重新创建图。** 这种设计是为了让forward pass中的运算过程可以使用任意Python control flow statements。因为一旦有了control flow statements，每次迭代中图的形状和大小就可能发生变化。每次迭代都重新创建图的好处是，在启动训练之前,不必编码所有可能的path。实际运行了哪些函数，最后就求哪些函数对应的梯度。

In [1]:
import torch

x = torch.ones(5)   # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)  # 要计算梯度
b = torch.randn(3, requires_grad=True)     # 要计算梯度
z = x @ w + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [2]:
# tensor的grad_fn属性中存放了ref to the BP function
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7f4e84715660>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7f4e847158d0>


### 计算梯度
1. 只有计算图中leaf nodes(requires_grad设为True时)才有grad properties，其他nodes都没有。
2. 一个计算图默认只能做一次BP。这是pytorch基于性能考虑设置的规则。如果要多次BP，要在backward call中设置参数retain_graph=True。

In [3]:
loss.backward()
print(w.grad)
print(b.grad)
print(x.grad)

tensor([[0.2144, 0.3061, 0.3074],
        [0.2144, 0.3061, 0.3074],
        [0.2144, 0.3061, 0.3074],
        [0.2144, 0.3061, 0.3074],
        [0.2144, 0.3061, 0.3074]])
tensor([0.2144, 0.3061, 0.3074])
None


### backward所需的数据如何传递
1. 自定义nn.autograd.Function时，在forward method中用save_for_backward()来存储backward中会用到的tensor data。这些数据在backward method中可以用saved_tensors() method提取。pytorch中已经定义好的函数也是通过这种方式向backward method传递data的。

2. 在debug需要的时候，可以查看grad_fn对应的saved tensors。这些tensor以grad_fn的attribute形式存在，attribute name以'_saved'开头。
3. pytorch在saving时，会将function中的saved tensor打包，在读取他们的时候再。unpack。但在读取他们的时候可能会因为要防止reference cycles，将tensor unpack成另一个tensor。此时读取出来的tensor和原来的tensor很可能不是同一个tensor。但是他们有共同的data storage。2.1版本中，只有tensor是其对应的grad_fn的output，才会这样。但这个implementation dettails在未来可能会变化。
4. 可以通过hooks for saved tensors来控制pytorch如何packing/unpacking saved tensors.

In [4]:
# save input of function
x = torch.randn(5, requires_grad=True)
y = x.pow(2)
print(x.equal(y.grad_fn._saved_self)) #
print(x is y.grad_fn._saved_self)     #

True
True


In [5]:
# save output of function
x = torch.randn(5, requires_grad=True)
y = x.exp()
print(x.equal(y.grad_fn._saved_result)) #
print(x is y.grad_fn._saved_result)     #

False
False


## locally disabling gradient tracking/computation
默认所有requires_grad=True的tensor都会被track computational history。但有的时候，不需要计算梯度，为了提高运算效率，可以disable gradient computation。

· **不需要做gradient computation的典型场景：** \
  (1) model已经train好了，只做inference，不需要好用资源track history\
  (2) frozen参数来做finetune

· **diable gradient tracking的两类方法：** \
  (1) disable entire block of code用context manager,比如：no-grad mode和inference mode\
  (2) 在更精细的范围内exclusion of sub-gragphs from gradient computation可以通过设置tensor的requires_grad值。

### 方法1：setting requires_grad
1. tensor的requires_grad可以用来控制forward和backward中的gradient computation是否发生。requires_grad的默认是false，但wrapped in nn.Parameter的tensor的默认值是True。
(1)Forward pass中，只有当operation中有tensor的requires_grad=True的时候才会被record到backward graph上。
(2)Backward pass: 只有requires_grad=True的leaf tensor才会计算gradients并累积到tensor.grad上。所以，尽管每个tensor都设定了requires_grad的值，实际上只有leaf tensor的这个设定是有意义的。
2. leaf tensor和non-leaf tensor的区别：
(1)leaf: 没有grad_fn; 没有backward graph与之对应；requires_grad的值要设定，要么手动设定，要么nn.Parameter中自动设定。
(2)non-leaf:有grad_fn; 有backward graph与之对应，因此他们的gradient只是作为intermediate result来计算requires_grad=True的leaf tensor的gradient；也因此，所有non-leaf tensor都自动设定了requires_grad=True，不需要手动设定。

### 方法2：setting grad mode
· pytorch中有3种grad model，他们决定了autograd处理gradient的方式：默认的是grad mode，此外还有no-grad mode和inference mode。他们都可以通过context manager和decorator来设置。
1. **<font color=red>grad mode</font>:** 只有在该mode下，requires_grad的配置才能起作用，autograd才会record operations on the tensors。在另外两种mode下，requires_grad都会被overriden to be False。

2. **<font color=red>no-grad mode</font>:** 让autograd暂时不记录operations（不将operations记录到backward graph上），退出该mode回到grad mode后再根据requires_grad的值是True还是False决定是否记录。\
  **· <font color=orange>适用场景：</font>** 只是暂时中断tensor operation的autograd，但tensor在当前operation中的运算结果还会被用于grad mode下的operation。\
  **· <font color=lightgreen>例子1, optimizer: </font>** 此时，weights' update不需要被autograd记录下来，但更新后的weights的值要用到下一次迭代中。而参与下一次迭代的forward运算时，这些weights是在grad mode下作计算的。\
  **· <font color=lightgreen>例子2, torch.nn.init</font>** 初始化operation是不需要计算梯度的。但是初始化完成后的training过程中，这些weights执行forward运算都是在grad mode中。\

3. **<font color=red>inference mode</font>:** 不是暂停让autograd记录tensor参与的operation，而是完全关闭autograd对operation的tracking。优点是tensor的运算速度会比no-grad mode更快。代价是inference mode下创建的tensor即使退出inference mode后，autograd也无法tracking它们参与的operations。\
  **· <font color=orange>适用场景：</font>** tensor参与的运算不需要记录到backward graph，并且这些运算中创建的tensor后续也不会用于其他需要被autograd记录的运算中去。\
  **· <font color=lightgreen>例子1, data processing: </font>** 不是模型的一部分，不做backward。\
  **· <font color=lightgreen>例子2, model evaluation：</font>** inference不做backward\

4. **<font color=red>evaluation mode</font>:** 这个不是disable autograd的机制，但容易和上述机制混淆。\
从功能上看，module.eval()和前面的no-grad和inference mode完全是不同的东西。理论上，model.eval()的作用取决于model中使用的modules，以及他们是否定义了trainning specific behavior。\
比如：model中用了torch.nn.Dropout或者torch.nn.Batchnorm，他们的特点是在trainning和validation/test时执行的operation不一样，这种情况下，就要手动在trainning和val/test阶段分别调用model.eval(), model.train()。
tensor的detach method。\
· <font color=red>建议：不论有没有定义trainning specific behavior，都在trainning和val/test阶段分别调用model.eval(), model.train()。因为，model中使用的module即使在现在没有trainning specific behavior，未来可能会发生改变。</font>

In [6]:
## 不区分trainning和val/test (伪码)

# for t in range(1000):
#   y_pred_train = model(x_train)
#   loss = loss(y_pred_train, y_train)
#   loss.backward()
#   opt.step()
#   opt.zero_grad()

#   with torch.no_grad():
#     y_pred_val = model(x_val)
#     loss = loss(y_pred_val, y_val)
#     val_loss = sum(loss) / len(y_val)

In [7]:
## 区分trainning和val/test (伪码)

# for t in range(1000):
#   model.train()
#   y_pred_train = model(x_train)
#   loss = loss(y_pred_train, y_train)
#   loss.backward()
#   opt.step()
#   opt.zero_grad()

#   model.eval()
#   with torch.no_grad():
#     y_pred_val = model(x_val)
#     loss = loss(y_pred_val, y_val)
#     val_loss = sum(loss) / len(y_val)

In [8]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [9]:
z = torch.matmul(x, w)+b
print(z.requires_grad)
z_det = z.detach()
print(z_det.requires_grad)

True
False


## directed acyclic graph(DAG)
1. DAG由Function对象组成
2. 叶节点是input tensor，根节点是output tensor，通过从root到leaves遍历graph，可以用chainrule自动计算gradients
3. forward pass中，autograd会同时做两件事：\
(1)执行forward operation，计算tensor\
(2)将operation的gradient function存到DAG中
4. 当.backward()被调用的时候，就会做backward pass。此时autograd会做3件事：
(1)用每个.grad_fn计算梯度\
(2)将梯度累积到各自对应tensor的.grad属性中\
(3)使用chain rule，向leaf tensor传梯度

# 自定义torch.autograd

步骤：\
· step1: subclass torch.autograd.Function, implement forward(), setup_context(), and backward() methods. \
· step2: 在以ctx（context object）作为argument的method中设置好其内容。\
· step3: declare是否支持double backwards\
· step4: 做gradient check

### step1：subclass Function
1. forward
2. ctx可以在forward中通过将ctx作为argument来对其设置。如果forward中没有设置ctx，就要用setup_context()。setup_context()只负责设置ctx，不做其他运算。
3. backward定义了gradient formula。\
· backward() method的tensor arguments是upstream gradients，function的每个output对应1个upstream gradient所以backward函数的tensor arguments数量与本函数的output数量相同。\
· backward() method的输出是function inputs的gradient。所以，backward的output tensor数量就是function inputs的数量。如果有的input tensor不用求gradient，或者有的input不是tensor，也不用求gradient，那么它们对应的输出用python：None。如果forward()中有optional arguments，那么输出的数量可能超过function input的数量，此时，backward()输出的数量也会超过function inputs，但这些forward arguments对应在backward中的output都要取None。

### step2：use the function in ctx properly,确保Funtction在autograd engine中正确工作
1. ctx.save_for_backward():存储backward pass中要用的tensor。\
· 如果要存储的data不是tensor，只能直接用ctx\
· 如果要存储的tensor既不是function的input也不是output，那么function就可能不支持double backward
2. ctx.mark_dirty()：用来mark在backward pass中会发生in-place modify的function input。
3. ctx.mark_none_differentiable()：用来标记function outputs是否需要求梯度。默认所有differentiable output tensors都设置为requires gradient。而不可导的那些则设置为不会被设置为requiures gradient。
4. ctx.set_materialize_grads()：当output tensor的计算不依赖于input值时，可以通过not materialize grad tensors given to backward function来优化梯度的计算方式。默认设置是Ture。如果设置为False，python中的None不会在call backward之前被转化为全0的tensor，此时，要手动处理这些None objects。

### step3：如果自定义的Function不支持double backward，要用decorating with once_differentiable()来声明
### step4：用torch.autograd.gradcheck()验证梯度计算的正确性。方法是用backward function计算Jacobian matrix，将结果与使用数值计算的结果做比较

In [10]:
## 例1：自定义LinearFunction
import torch
from torch.autograd import Function
class LinearFunction(Function):
  @staticmethod
  def forward(input, weight, bias):
    output = input @ weight.T
    if bias is not None:
      output += bias.unsqueeze(0).expand_as(output)
    return output

  @staticmethod
  # inputs是所有传给forward()的inputs构成的tuple
  # output是forward()的output
  def setup_context(ctx, inputs, output):
    input, weight, bias = inputs
    ctx.save_for_backward(input, weight, bias)

  @staticmethod
  # grad_output是backward()唯一的输出
  def backward(ctx, grad_output):
    # unpack所有的saved_tensors
    input, weight, bias = ctx.saved_tensors
    # 初始化所有inputs的gradient为None，因为additional trailing Nones are ignored
    # 因此，即使在Function有additional input的时候，return statement都可以很简单
    grad_input = grad_weight = grad_bias = None

    # 增加的条件验证只是为了避免不必要的运算
    if ctx.needs_input_grad[0]:
      grad_input = grad_output.mm(weight)
    if ctx.needs_input_grad[1]:
      grad_weight = grad_output.t().mm(input)
    if bias is not None and ctx.needs_input_grad[2]:
      grad_bias = grad_output.sum(0)

    return grad_input, grad_weight, grad_bias


In [11]:
## 应用方式
#  1. 最好wrapped into function形式
def linear(input, weight, bias=None):
    return LinearFunction.apply(input, weight, bias)

#  2. 直接用
linear = LinearFunction.apply

In [12]:
## 例2：自定义一个有non-tensor argument的函数
class MulContent(Function):
  @staticmethod
  def forward(tensor, constant):
    return tensor * constant

  @staticmethod
  # inputs是所有传给forward()的inputs构成的tuple
  # output是forward()的output
  def setup_context(ctx, inputs, output):
    tensor, constant = inputs
    ctx.constant = constant  # 以ctx.constant的形式存非tensor

  @staticmethod
  # grad_output是backward()唯一的输出
  def backward(ctx, grad_output):    
    return grad_output * ctx.constant, None # constant的梯度返回值用None

In [13]:
## 例3（优化上例）：calling set_materialize_grads(False):
class MulContent(Function):
  @staticmethod
  def forward(tensor, constant):
    return tensor * constant

  @staticmethod
  # inputs是所有传给forward()的inputs构成的tuple
  # output是forward()的output
  def setup_context(ctx, inputs, output):
    tensor, constant = inputs
    ctx.set_materialize_grads(False)
    ctx.constant = constant  # 以ctx.constant的形式存非tensor

  @staticmethod
  # grad_output是backward()唯一的输出
  def backward(ctx, grad_output):
    # must handle None grad_output tensor
    # 这样就可以skip不必要的计算，直接返回None
    if grad_output is None:
        return None, None
    
    return grad_output * ctx.constant, None

如果需要save在forward中计算的intermediate tenosr，他们要么returned as output，要么combine forward和setup_context()。
这意味着如果想要通过这些intermediate values来计算gradient，就要为这些intermediate values定义gradient formula

In [14]:
## 例4：define gradient formula for intermediate values
class MyCube(Function):
  @staticmethod
  def forward(x):
    # 需要save x for backward，所以要将其return as output
    dx = 3 * x ** 2
    result = x ** 3
    return result, dx

  @staticmethod
  # inputs是所有传给forward()的inputs构成的tuple
  # output是forward()的output
  def setup_context(ctx, inputs, output):
    x, = inputs          # 因为inputs是tuple，所以只有1个元素的时候也要用'x,'
    result, dx = output
    ctx.save_for_backward(x, dx)

  @staticmethod
  # grad_output是backward()唯一的输出
  def backward(ctx, grad_output): 
    x, dx = ctx.saved_tensors
    result = grad_output * dx + grad_dx * 6 * x    
    return result

# wrap MyCube in a function
def my_cube(x):
    result, dx = MyCube.apply(x)
    return result